<h1><center> Projeto PCD </center></h1>
<center> Autores: Bruna Guedes; Jônatas de Oliveira; Luiza Davoli; Matheus Macedo </center>

## Sumário

1. Introdução  
2. Bibliotecas

3. Funções

    3.1. numero_genomas

    3.2. seleção_virus

    3.3. buscar_genoma

    3.4. baixar_genoma

    3.5. rodar_muscle

    3.6. plotar_matriz_triangular

4. O Código

5. Agradecimentos

6. Referências  


## Introdução

Projeto realizado para a diciplina de **Práticas em Ciência de Dados** no primeiro semestre de 2025 do curso de Ciência e Tecnologia, **Ilum Escola de Ciência**.

A ideia para o projeto surgiu do desejo de relacionar conceitos estudados em outras áreas do conhecimento com as funcionalidades do Python estudadas durante o primeiro semestre. Por esse motivo, optou-se por aplicar as ideias de grafos, matrizes e processamento de strings para comparar as semelhanças e diferenças entre genomas de vírus já catalogados.

## Bibliotecas

Abaixo estão listadas todas as bibliotecas utilizadas em nosso código:

In [2]:
from Bio import Entrez, SeqIO
from pathlib import Path
import subprocess
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
from tkinter import *
from tkinter import ttk

#### Biopython

É um conjunto de ferramentas gratuitas para computação biológica em Python. É projeto membro da Open Bioinformatics Foundation (OBF), com repositório no GitHub (https://github.com/biopython).


Ele é responsável por manter diversos recursos biológicos, incluindo o GenBank, um repositório público de sequências de nucleotídeos e suas anotações. Este banco de dados é amplamente utilizado em bioinformática para análise de genomas, sendo uma fonte essencial de dados para ferramentas de busca, comparação e alinhamento de sequências genéticas.

Entrez

O Entrez é o sistema de busca e integração de dados do NCBI, que permite o acesso a diversos bancos de dados biológicos, como o GenBank, PubMed, Gene, entre outros.

Neste projeto, utilizamos o Entrez para acessar e baixar os genomas de variantes virais diretamente do NCBI. Isso foi feito por meio do módulo Bio.Entrez, fornecido pelo pacote Biopython, que oferece uma interface programática para as funcionalidades disponibilizadas pelo NCBI.

Mais especificamente, acessamos o banco de dados GenBank para obter as sequências genômicas necessárias para análise.

Muscle

Multiple Sequence Comparison by Log-Expectation é um software de computador para alinhamento de múltiplas sequências de proteínas e nucleotídeos, sendo licenciado como domínio público.

O alinhamento de sequências é o processo de comparar duas sequências (no caso, de nucleotídeos) para observar seu nível de identidade. O alinhamento entre duas sequências pode ser feito de forma global ou local. Quando global, comparamos uma sequência, ao longo de toda sua extensão, mas local se a comparação é apenas em pequenas regiões. Nesse contexto, MUSCLE é capaz de comparar genomas de ambas as formas, retornando um score de alinhamento, que é a soma dos pontos de similaridades, diferenças e abertura e extensão de falhas.

O alinhamento é considerado simples quando apenas duas sequências são alinhadas, e múltipla quando três ou mais devem ser alinhadas entre si. No caso, a depender das entradas fornecidas pelo usuário, alinhamento simples ou múltiplo será realizado.

SeqIO

É um módulo da biblioteca Biopython para ler e escrever sequências de DNA, RNA ou proteínas a partir de arquivo como FASTA, GenBank ou EMBL, além de converter um formato para outro, lidando com arquivos contendo uma ou mais sequências representadas como objetos SeqRecord.

AlignIO

É um módulo de Biopython que fornece uma interface de entrada e saída de alinhamento de sequência, lidando com arquivos que contêm um ou mais alinhamentos de sequência representados como objetos Alignment.

Subprocess

O módulo subprocess é parte da biblioteca padrão do Python. É usado para executar comandos do sistema operacional a partir do seu script Python. No caso, foi utilizado para chamar o MUSCLE.exe .

matplotlib.pyplot

O módulo pyplot, parte da biblioteca Matplotlib, é utilizado para criar gráficos e visualizações em Python.

Seaborn

É uma biblioteca de visualização de dados em python que deixa os gráficos mais profissionais e informativos, usando como base o matplotlib. No caso, foi utilizado para gerar, com base na matriz de similaridade, um heatmap (gráfico em forma de tabela colorida para visualizar valores numéricos, onde as cores representam os números).

Numpy

NumPy (Numerical Python) é uma biblioteca do Python usada para fazer cálculos matemáticos e trabalhar com arrays (matrizes/vetores). No caso, utilizamos NumPy para criar a matriz de similaridade entre genomas antes de exibir no heatmap com seaborn (função calcular_similaridade), além de plotar a matriz triangular (função plotar_matriz_triangular).

Plopy

## Funções

Abaixo definimos todas as funções necessárias para o funcionamento do código:

`numero_genomas` : Essa função cria uma caixa interativa, onde é possível digitar a quantidade de genomas que se deseja comparar

In [6]:
def numero_genomas():

    # Esse bloco cria e configura a janela que será aberta pelo tkinter
    janela = Tk()
    janela.title('Painel interativo')
    janela.geometry('600x100+100+100')
    janela.attributes('-topmost', 1)
    conteudo = ttk.Frame(janela, padding=(3,12))
    titulo = ttk.Label(conteudo, text="Número de genomas que se deseja comparar")

    # Permite que o usuário digite em uma caixa de texto, realizando um input
    caixa_texto = ttk.Entry(conteudo)

    # Cria botões interativos com as funções desejadas
    def comando():
        numero = caixa_texto.get()
        janela.destroy()

        return numero
    
    ok = ttk.Button(conteudo, text="OK", command=comando)

    # Ajusta visualmente o tamanho e distância de cada componente da janela
    conteudo.grid(column=0, row=0, sticky=(N, S, E, W))
    titulo.grid(column=2, row=0, columnspan=4, sticky=(N, W), padx=5)
    caixa_texto.grid(column=0, row=1, columnspan=6, sticky=(N,E,W), pady=5, padx=5)
    ok.grid(column=4, row=3)

    janela.columnconfigure(0, weight=2)
    janela.rowconfigure(0, weight=2)
    conteudo.rowconfigure(1, weight=2)
    conteudo.columnconfigure(0, weight=4)
    conteudo.columnconfigure(1, weight=4)
    conteudo.columnconfigure(2, weight=4)
    conteudo.columnconfigure(3, weight=2)
    conteudo.columnconfigure(4, weight=2)

    # Executa o código, permitindo que a janela seja aberta
    janela.mainloop()

    return numero

`seleção_virus` : Essa função cria uma caixa interativa, onde é possível digitar o nome do vírus que se deseja procurar o genoma. A variável `nome_virus` salva os vírus selecionados, para que se possa realizar a busca no NCBI.

In [11]:
def seleção_virus():

    # Esse bloco cria e configura a janela que será aberta pelo tkinter
    janela = Tk()
    janela.title('Painel interativo')
    janela.geometry('600x100+100+100')
    janela.attributes('-topmost', 1)
    conteudo = ttk.Frame(janela, padding=(3,12))
    titulo = ttk.Label(conteudo, text="Nome do Vírus")

    # Permite que o usuário digite em uma caixa de texto, realizando um input
    caixa_texto = ttk.Entry(conteudo)

    # Cria botões interativos com as funções desejadas
    def comando():
        nome_do_virus = caixa_texto.get()
        janela.destroy()

        return nome_do_virus
    
    ok = ttk.Button(conteudo, text="OK", command=comando)

    # Ajusta visualmente o tamalho e distância de cada componente da janela
    conteudo.grid(column=0, row=0, sticky=(N, S, E, W))
    titulo.grid(column=2, row=0, columnspan=4, sticky=(N, W), padx=5)
    caixa_texto.grid(column=0, row=1, columnspan=6, sticky=(N,E,W), pady=5, padx=5)
    ok.grid(column=4, row=3)

    janela.columnconfigure(0, weight=2)
    janela.rowconfigure(0, weight=2)
    conteudo.rowconfigure(1, weight=2)
    conteudo.columnconfigure(0, weight=4)
    conteudo.columnconfigure(1, weight=4)
    conteudo.columnconfigure(2, weight=4)
    conteudo.columnconfigure(3, weight=2)
    conteudo.columnconfigure(4, weight=2)

    # Executa o código, permitindo que a janela seja aberta
    janela.mainloop()

    return nome_virus

`buscar_genoma` : Essa função busca o vírus selecionado na base de dados do NCBI (National Center for Biotechnology Information). Para isso, a função recebe o nome do vírus, busca o ID correspondente e retorna essa informação ao usuário. Esse é um passo muito importante, pois um mesmo vírus pode ter diferentes cepas e sequenciamentos, os quais terão diferentes IDs no NCBI. Caso o vírus não esteja na base de dados do site, a função informa o usuário sobre esse problema e requisita uma nova tentativa

In [12]:
def buscar_genoma(nome_virus):
    Entrez.email = "computandocombiopy@gmail.com"
    termo_de_busca = f"{nome_virus} complete genome"
    busca = Entrez.esearch (db = "nucleotide", term = termo_de_busca, retmax = 1)
    resultado = Entrez.read(busca)
    busca.close()
    
    id_do_virus = (resultado["IdList"])

    if not id_do_virus:
        print(f"Não há um vírus {nome_virus} no NCBI. Tente outra vez!")

    return id_do_virus[0]

`baixar_genoma` : Essa função será utilizada para baixar os genomas dos vírus escolhidos pelo usuário. 

In [13]:
def baixar_genoma(iden, output_folder=Path("data")/"genomes"):
    
    # Faz os genomas irem para uma pasta relativa ao repositório
    output_folder.mkdir(parents=True, exist_ok=True)
    filename= output_folder/f"{iden}.fasta"
    
    # Acessando o Entrez
    Entrez.email="computandocombiopy@gmail.com"

    # Checa se o arquivo fasta existe, se não, ele o cria
    if not filename.exists():
        
        #Baixa o arquivo .fasta (funciona para GenBank e RefSeq)
        stream= Entrez.efetch(
            db="nucleotide",id=iden, rettype="fasta", retmode="text"
        )
        data = stream.read()
        
        #Checa se data tá vazio ou não
        if not data:
            print("Error: Resposta vazia do NCBI.")
            
        else:
            with open(filename, "w") as output:
                output.write(data)
        print(f"Salvo em {filename}")

`rodar_muscle` : A função recebe 2 arquivos como parâmetros, um `genomas.fasta` e outro `genomas_alinhados.fasta`. Depois, roda o muscle, utilizado para realizar o alinhamento entre genomas de diferentes vírus e compará-los. Esse alinhamento é salvo no arquivo `genomas_alinhados.fasta`.

In [14]:
def rodar_muscle(entrada_fasta, saida_fasta): 
    
    # Acessa o executável do MUSCLE
    muscle_exe = "muscle-win64.v5.3.exe"
    
    # Alinha o arquivo de entrada e coloca os dados de saída no arquivo criado pelo executável
    comando = [muscle_exe, "-align", entrada_fasta, "-output", saida_fasta]
    
    # Tratamento de erro
    try: 
        resultado = subprocess.run(comando, check=True, capture_output=True, text=True) 
        print("MUSCLE finalizado com sucesso.")
        print("STDOUT:", resultado.stdout)
        print("STDERR:", resultado.stderr)
    except subprocess.CalledProcessError as e:
        print("Erro ao rodar MUSCLE:")
        print("STDOUT:", e.stdout)
        print("STDERR:", e.stderr)
        
        # Imprime o erro novamente, além de interromper o programa, impedindo que o erro se prolongue
        raise 

`plotar_matriz_triangular` : A função recebe uma lista de nomes e uma matriz de similaridade, com valores entre 0 e 1. Cria uma matriz triangular, representando as semelhanças e diferenças entre os vírus escolhidos. Com isso, gera um gráfico, possibilitando um entendimento mais visual.

Definindo melhor alguns dos termos utilizados nessa função:

- A **máscara** para esconder a parte superior da matriz (acima da diagonal) é útil porque a matriz é simétrica — ou seja, o valor de similaridade de A com B é o mesmo que de B com A. Dessa forma, usar a máscara evita mostrar dados duplicados visualmente.

- `np.ones_like(matriz_np, dtype=bool)` cria um array (no caso, uma matriz) com o mesmo formato da matriz_np, preenchidos com valor 1. Entretanto, como utilizamos dtype = bool, 1 é convertido para True. Portanto, a matriz é toda preenchida com True.

- `np.triu` significa "upper triangular". Dessa forma, essa função recebe uma função e retorna a parte superior dela com os valores originais e a inferior como false. Como a matriz é toda True, ela mantém True na parte triangular superior e define False na parte inferior.

- Os valores númericos da matriz matriz_np que estão na máscara como False serão exibidos no gráfico.

- `annot = True` exibe os valores numéricos da matriz dentro de cada célula do gráfico.

- xticklabels=nomes os nomes aparecem no eixo x, sendo que cada coluna recebe um nome de virus.

- yticklabels=nomes os nomes também aparecem no eixo y, sendo que cada linha recebe um nome de virus.

In [15]:
def plotar_matriz_triangular(nomes, matriz):

    # Garante que a matriz é um array NumPy, necessário para facilitar as operações matemáticas e criação de gráficos
    matriz_np = np.array(matriz) 
    
    # Máscara para esconder a parte superior da matriz (acima da diagonal)
    matriz_bool = np.ones_like(matriz_np, dtype=bool)
    mascara = np.triu(matriz_bool)
    
    # Módulo para criar uma figura com tamanho definido em polegadas (largura, altura)
    plt.figure(figsize=(8, 6)) 
    sns.heatmap(matriz_np, annot=True, fmt=".2f", mask=mascara,
                xticklabels=nomes, yticklabels=nomes,
                cmap="YlGnBu", linewidths=0.5, cbar_kws={"label": "Similaridade"},
                vmin=0.001, vmax=1.0)

    plt.title("Matriz de Similaridade Genômica", fontsize=16)
    plt.xticks(rotation=45, ha='right') 
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()

## Código

In [7]:
numero_genomas()

NameError: name 'numero' is not defined

## Agradecimentos

Agradecemos muito aos professores dessa disciplina, por todo o aprendizado durante o semestre e a colaboração para a realização desse projeto:

Professor Daniel Roberto Cassar

Professor James Moraes de Almeida

Professor Leandro Nascimento Lemos

E à professora Juliana Helena Costa Smetana, da disciplina de Energia em Sistemas Vivos I, pela colaboração com as ideias iniciais, os conselhos e o apoio durante a realização de todo o projeto.

## Referências

Sequenciamento genômico: no rastro do vírus. Disponível em: <https://agencia.fiocruz.br/sequenciamento-genomico-no-rastro-do-virus>. Acesso em: 11 jun. 2025.

Accessing NCBI’s Entrez databases — Biopython 1.85 documentation. Disponível em: <https://biopython.org/docs/latest/Tutorial/chapter_entrez.html#chapter-entrez>. Acesso em: 11 jun. 2025.

NATIONAL CENTER FOR BIOTECHNOLOGY INFORMATION. Our Mission - NCBI. Disponível em: <https://www.ncbi.nlm.nih.gov/home/about/mission/>.

‌DOS, C. GenBank. Disponível em: <https://pt.wikipedia.org/wiki/GenBank>. Acesso em: 11 jun. 2025.

‌BIOPYTHON. Biopython · Biopython. Disponível em: <https://biopython.org/>.

Biopython Documentation — Biopython 1.85 documentation. Disponível em: <https://biopython.org/docs/latest/index.html#>. Acesso em: 11 jun. 2025.

‌Cock, P. J. A., Antao, T., Chang, J. T., Chapman, B. A., Cox, C. J., Dalke, A., Friedberg, I., Hamelryck, T., Kauff, F., Wilczynski, B., & de Hoon, M. J. L. (2009). Biopython: freely available Python tools for computational molecular biology and bioinformatics. Bioinformatics, 25(11), 1422–1423.

‌OPENAI. ChatGPT (versão GPT-4) [ferramenta de inteligência artificial]. Disponível em: https://chat.openai.com/. Acesso em: 11 jun. 2025.

WIKIPEDIA CONTRIBUTORS. Sequence alignment. Disponível em: <https://en.wikipedia.org/wiki/Sequence_alignment>.

WIKIPEDIA CONTRIBUTORS. MUSCLE (alignment software).

INTRODUÇÃO, 3. 1. Alinhamento de Seqüências. Disponível em: <https://professor.pucgoias.edu.br/SiteDocente/admin/arquivosUpload/18497/material/Cap.%203%20Alinhamento%20de%20sequ%C3%AAncias.pdf>. Acesso em: 11 jun. 2025.

The module for multiple sequence alignments, AlignIO. Disponível em: <https://biopython.org/wiki/AlignIO>. Acesso em: 11 jun. 2025.

Introduction to NumPy. Disponível em: <https://www.w3schools.com/python/numpy/numpy_intro.asp>. Acesso em: 11 jun. 2025.



Edgar, R. C. (2004). MUSCLE: a multiple sequence alignment method with reduced time and space complexity. BMC Bioinformatics, 5, 113. 


TkDocs home. Disponível em: <https://tkdocs.com/index.html>. Acesso em: 18 jun. 2025.